In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import ColumnDataSource, figure, output_file, save
from bokeh.io import show
from bokeh.models import FactorRange, Legend
from bokeh.palettes import Category10

In [3]:
data = pd.read_csv(r"C:\Users\inest\OneDrive - Danmarks Tekniske Universitet\Semester IV\Social Data Analysis and Visualization\merged_data.csv")

In [4]:
data.head()

,Category,PdDistrict,Longitude,Latitude,TimeOfDayHour,TimeOfDayMinute,DayOfWeek,DayOfMonth,Month,Year
0,ROBBERY,INGLESIDE,-122.420084,37.708311,17,50,Monday,22,November,2004
1,VEHICLE THEFT,PARK,-120.500000,90.000000,20,0,Tuesday,18,October,2005
2,VEHICLE THEFT,SOUTHERN,-120.500000,90.000000,2,0,Sunday,15,February,2004
3,ASSAULT,SOUTHERN,-122.410541,37.770913,17,0,Sunday,21,November,2010
4,ASSAULT,TARAVAL,-122.470366,37.745158,15,50,Tuesday,2,April,2013


In [ ]:
focus_crimes = ['WEAPON LAWS', 'PROSTITUTION', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUG/NARCOTIC', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY']

In [15]:
data = data[data['Category'].isin(focus_crimes)]
hourly_data = data.groupby(['TimeOfDayHour', 'Category']).size().unstack()

In [16]:
hourly_data.head()

Category,ASSAULT,BURGLARY,DRUG/NARCOTIC,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,VANDALISM,VEHICLE THEFT,WEAPON LAWS
TimeOfDayHour,,,,,,,,,,
0,11632,6412,4324,30342,1466,3764,762,9968,6763,1711
1,10532,4263,2622,17246,1114,3843,496,6613,4295,1313
2,9333,5072,2169,11536,833,3875,439,6255,3512,1169
3,4659,5786,1497,8069,583,2354,339,4642,2392,769
4,3111,5580,1061,5700,485,1570,314,3416,1884,525


In [17]:
#normalize the data by hour and category
hourly_data = hourly_data.div(hourly_data.sum(axis=1), axis=0)
hourly_data.head()

Category,ASSAULT,BURGLARY,DRUG/NARCOTIC,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,VANDALISM,VEHICLE THEFT,WEAPON LAWS
TimeOfDayHour,,,,,,,,,,
0,0.150783,0.083117,0.056051,0.393316,0.019003,0.048792,0.009878,0.129213,0.087667,0.022179
1,0.201234,0.081453,0.050098,0.329518,0.021285,0.073428,0.009477,0.126354,0.082064,0.025087
2,0.211187,0.114769,0.049080,0.261037,0.018849,0.087684,0.009934,0.141538,0.079470,0.026452
3,0.149855,0.186105,0.048151,0.259537,0.018752,0.075716,0.010904,0.149308,0.076938,0.024735
4,0.131566,0.235981,0.044870,0.241056,0.020511,0.066396,0.013279,0.144464,0.079675,0.022202


In [44]:

# Define a color palette
colors = Category10[len(focus_crimes)]

hour = [str(i) for i in range(24)]
src = ColumnDataSource(hourly_data)

p = figure(x_range=FactorRange(factors=hour), title="Normalized number of incidents by hour")
bar = {}  # to store vbars

# Create a vbar for each focus crime with a specific color
for indx, i in enumerate(focus_crimes):
    bar[i] = p.vbar(x='TimeOfDayHour', top=i, source=src, color=colors[indx],
                    legend_label=i, muted_alpha=0.1, muted=True)

In [45]:
# Remove the default legend
p.legend.visible = False

# Add custom legend
legend = Legend(items=[(x, [bar[x]]) for x in focus_crimes], location=(0, -30), click_policy='mute')
p.add_layout(legend, 'right')  # assigns the click policy (you can try to use ''hide')
show(p)

In [48]:
output_file("_assets/bokeh_plot.html")
save(p)

'c:\\Users\\inest\\OneDrive - Danmarks Tekniske Universitet\\Semester IV\\Social Data Analysis and Visualization\\Assignment 2\\inestacanho.github.io\\_assets\\bokeh_plot.html'